## Введение в обработку естественного языка
### Урок 9. Языковое моделирование
### Практическое задание

**Взять набор данных на ваше усмотрение (стихи/прозу) или что-то ещё для примера можно так же использовать прикреплённый Евгений Онегин**
<ol>
<li>поэкспериментировать с посимвольным подходом</li>
<li>проверить насколько изменится качество генерации текста при токенизации по словам</li>
</ol>

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !ls

In [4]:
# !ls

In [5]:
path_to_file = 'Busido.txt'

In [10]:
text = open(path_to_file, 'rb').read().decode(encoding='Windows-1251')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 495600 characters


In [11]:
print(text[:250])

Бусидо. Военный канон самурая с комментариями
Ямамото Цунэтомо

Миямото Мусаси

Юдзан Дайдодзи


Иллюстрированная военная история
Первые самураи появились в Японии еще в VII в. Шло время, сформировались устои морального кодекса самурая, позж


In [12]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

118 unique characters


In [13]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [14]:
text_as_int, text, len(text_as_int), len(text)

(array([52, 99, 97, ...,  1,  2,  1]),
 'Бусидо. Военный канон самурая с комментариями\r\nЯмамото Цунэтомо\r\n\r\nМиямото Мусаси\r\n\r\nЮдзан Дайдодзи\r\n\r\n\r\nИллюстрированная военная история\r\nПервые самураи появились в Японии еще в VII\xa0в. Шло время, сформировались устои морального кодекса самурая, позже превратившегося в свод заповедей «Путь воина» («Бусидо»). Века сменялись тысячелетиями, но и по сей день моральный кодекс самураев не утратил своей актуальности. В этой книге собраны наиболее авторитетные трактаты и руководства, посвященные бусидо. «Будосёсинсю» Юдзана Дайдодзи, «Хагакурэ» Ямамото Цунэтомо и «Книга пяти колец» Миямото Мусаси непременно станут духовными спутниками каждого, кто ищет ответы на главные вопросы в своей жизни.\r\n\r\nВ формате a4.pdf сохранен издательский макет.\r\n\r\n\r\n\r\n\r\n\r\nЯмамото Цунэтомо, Миямото Мусаси, Юдзан Дайдодзи\r\n\r\nБусидо. Военный канон самурая с комментариями\r\n\r\n\r\n\r\n\r\n\r\nЮдзан Дайдодзи. Будосёсинсю\r\n\r\n\r\n\r\n

### train and target

In [15]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Б
у
с
и
д


In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Бусидо. Военный канон самурая с комментариями\r\nЯмамото Цунэтомо\r\n\r\nМиямото Мусаси\r\n\r\nЮдзан Дайдодзи\r\n'
'\r\n\r\nИллюстрированная военная история\r\nПервые самураи появились в Японии еще в VII\xa0в. Шло время, сформ'
'ировались устои морального кодекса самурая, позже превратившегося в свод заповедей «Путь воина» («Бус'
'идо»). Века сменялись тысячелетиями, но и по сей день моральный кодекс самураев не утратил своей акту'
'альности. В этой книге собраны наиболее авторитетные трактаты и руководства, посвященные бусидо. «Буд'


In [17]:
text

'Бусидо. Военный канон самурая с комментариями\r\nЯмамото Цунэтомо\r\n\r\nМиямото Мусаси\r\n\r\nЮдзан Дайдодзи\r\n\r\n\r\nИллюстрированная военная история\r\nПервые самураи появились в Японии еще в VII\xa0в. Шло время, сформировались устои морального кодекса самурая, позже превратившегося в свод заповедей «Путь воина» («Бусидо»). Века сменялись тысячелетиями, но и по сей день моральный кодекс самураев не утратил своей актуальности. В этой книге собраны наиболее авторитетные трактаты и руководства, посвященные бусидо. «Будосёсинсю» Юдзана Дайдодзи, «Хагакурэ» Ямамото Цунэтомо и «Книга пяти колец» Миямото Мусаси непременно станут духовными спутниками каждого, кто ищет ответы на главные вопросы в своей жизни.\r\n\r\nВ формате a4.pdf сохранен издательский макет.\r\n\r\n\r\n\r\n\r\n\r\nЯмамото Цунэтомо, Миямото Мусаси, Юдзан Дайдодзи\r\n\r\nБусидо. Военный канон самурая с комментариями\r\n\r\n\r\n\r\n\r\n\r\nЮдзан Дайдодзи. Будосёсинсю\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nПредисло

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [19]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Бусидо. Военный канон самурая с комментариями\r\nЯмамото Цунэтомо\r\n\r\nМиямото Мусаси\r\n\r\nЮдзан Дайдодзи\r'
Target data: 'усидо. Военный канон самурая с комментариями\r\nЯмамото Цунэтомо\r\n\r\nМиямото Мусаси\r\n\r\nЮдзан Дайдодзи\r\n'


In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [21]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [23]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 118) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           30208     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 118)           120950    
Total params: 10,387,062
Trainable params: 10,387,062
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ехотой при Фукусима Саэмон-тайю Масанори, день и ночь охранявший железные ворота замка Хиросима в Ак'

Next Char Predictions: 
 'ТmГ9e]рaжlГЖкб*и\xa0ю6*ЗАЮяъ!\rsIШЕС2дтA,ееСщ5Е…ч«Ож3зкц“ЯanЙf??*я05Э:Зm–вВ\tмДВt,ИiАtfнхаkЛnXёtЕ»шpФИфЮТ'


## Train the model

In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 118)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.770252


In [29]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [31]:
EPOCHS = 200

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
76/76 [==============================] - 631s 8s/step - loss: 3.2966
Epoch 2/200
76/76 [==============================] - 632s 8s/step - loss: 2.4085
Epoch 3/200
76/76 [==============================] - 790s 10s/step - loss: 2.1023
Epoch 4/200
76/76 [==============================] - 728s 10s/step - loss: 1.8509
Epoch 5/200
76/76 [==============================] - 846s 11s/step - loss: 1.6513
Epoch 6/200
 6/76 [=>............................] - ETA: 11:12 - loss: 1.5402

KeyboardInterrupt: 

## Generate text

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.0001

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Уж месяц "))